In [ ]:
# Load the temperature and humidity datasets
temp_data = pd.read_csv('../../data/input_synthetic_dataset/Tair_Tuddal_2019.csv')
rh_data = pd.read_csv('../../data/input_synthetic_dataset/RH_Tuddal_2019.csv')

# Ensure consistent datetime index
temp_data['time'] = pd.to_datetime(temp_data['time'])
rh_data['time'] = pd.to_datetime(rh_data['time'])
temp_data.set_index('time', inplace=True)
rh_data.set_index('time', inplace=True)

# Add temperature and humidity to the main data frame
data['temperature'] = temp_data['Tair']
data['humidity'] = rh_data['RH']

# Define parameters for melting
melting_temp_threshold = 273  # Melting occurs above 0°C
degree_day_factor = 3.0  # Melting rate in mm/°C/day, for simplicity
rain_melting_factor = 0.0125 # 0.5  # Rain contributes directly to melting

# Calculate melting rate
# Melting from temperature (only when temperature > 0°C)
data['temp_melt'] = data.apply(
    lambda row: max(row['temperature'] - melting_temp_threshold, 0) * row['time_delta'] / 86400 * degree_day_factor, axis=1
)

# Melting from rain (assuming rain contributes directly to melting)
data['rain_melt'] = data['Rainf'] * data['time_delta'] * rain_melting_factor

# Total melt
data['total_melt'] = data['temp_melt'] + data['rain_melt']

# Adjust cumulative snow cover by subtracting melt
data['adjusted_cumulative_snow_cover'] = (data['cumulative_snow_cover'] - data['total_melt'].cumsum()).clip(lower=0)

# Plotting the adjusted cumulative snow cover
plt.figure(figsize=(12, 6))
plt.plot(data.index, data['adjusted_cumulative_snow_cover'], label='Adjusted Cumulative Snow Cover (mm)', color='red')
plt.xlabel('Date')
plt.ylabel('Adjusted Cumulative Snow Cover (mm)')
plt.title('Adjusted Cumulative Snow Cover Over Time with Melting Considered in Tuddal 2019')
plt.legend()
plt.grid(True)
plt.show()